In [0]:
# creating a monitoring task for raw data gathering and creating dashboard

In [0]:
%sql
select * from mycatalog.hp_prd_data.raw_data limit 2

In [0]:
%sql
select
 processed_date,
 processed_hour,
 http_status_code,
 count(*) as total_rows,
 count(Distinct zipcode) as total_zipcodes
from mycatalog.hp_prd_data.raw_data
group by processed_date, processed_hour, http_status_code
order by processed_date desc

In [0]:
%sql
select
 count(*) as total_rows,
 count(Distinct source_url) as total_sources,
 count(Distinct zipcode) as total_zipcodes
from mycatalog.hp_prd_data.raw_data

In [0]:
# there are some duplicates do deduplication while transforming